### 코드 버전1 (에러있음)
xml.etree.ElementTree 모듈 사용관련
- https://docs.python.org/3/library/xml.etree.elementtree.html
- https://wikidocs.net/127832
- https://newly0513.tistory.com/178

In [ ]:
%pip install opencv-python

In [2]:
import os
import cv2 
import xml.etree.ElementTree as ET

def get_class_index(label):
    '''클래스 레이블을 클래스 인덱스로 매핑'''
    class_mapping = {'Hyoid_Bone': 0,
                     'Neck_Bone': 1, 'Food_Locate': 2, 'Food': 3}
    return class_mapping.get(label, 3)  # Default to 3 if label not found

In [19]:
xml_folder = './data/extract_img_test/extracted_img_231116_인덱싱에러수정(1)/25024694_현**_(1)_YP_인덱싱수정'
yolo_txt_dir = './data/yolo_label_test/yolo_output_231120'

In [ ]:
if not os.path.exists(yolo_txt_dir):
    os.makedirs(yolo_txt_dir)

for file in os.listdir(xml_folder):
    # 경로에 있는 xml 파일을 읽어 파싱
    if file.endswith('.xml'):
        print(file)
        tree = ET.parse(os.path.join(xml_folder, file))
        # XML 문서의 구조는 요소들의 계층 구조로 표현되며, 이 중 최상위에 위치하는 요소를 루트라고 하는데, getroot() 메서드를 사용하여 해당 루트에 직접 접근
        root = tree.getroot()
        for child in root:
            print("child.tag",child.tag)
            print("child.attrib", child.attrib)

In [ ]:
if not os.path.exists(yolo_txt_dir):
        os.makedirs(yolo_txt_dir) 
    
for file in os.listdir(xml_folder):
    # 경로에 있는 xml 파일을 읽어 파싱
    if file.endswith('.xml'):
        print(file)
        tree = ET.parse(os.path.join(xml_folder,file))
        # XML 문서의 구조는 요소들의 계층 구조로 표현되며, 이 중 최상위에 위치하는 요소를 루트라고 하는데, getroot() 메서드를 사용하여 해당 루트에 직접 접근
        root = tree.getroot()

        for image in root.iter('image'): # tag가 이미지면, iter
            # image 태그의 attrib에서 이름 추출
            image_name = image.attrib['name']
            print(image_name)
            label_name=image_name.replace('.jpg','.txt')
            print(label_name)

            width = float(image.attrib['width'])  # image 태그의 attrib에서 가로 
            height = float(image.attrib['height'])  # image 태그의 attrib에서 높이
            
            # 레이블 정보를 저장한 변수를 만든 후, 
            labels = '' # 레이블 초기화 

            # for loop 반복문을 돌면서,
            for box in image.iter('box'):
                # 바운딩 박스 'box'태그에서 'label'의 값을 읽어오고, cls로 표현
                label = box.attrib['label']
                cls = get_class_index(label)

                xtl = float(box.attrib['xtl'])
                ytl = float(box.attrib['ytl'])
                xbr = float(box.attrib['xbr'])
                ybr = float(box.attrib['ybr'])

                x_center = (xtl + (xbr - xtl) / 2) / width
                y_center = (ytl + (ybr - ytl) / 2) / height
                box_w = (xbr - xtl) / width
                box_h = (ybr - ytl) / height

                labels += f"{cls} {x_center} {y_center} {box_w} {box_h}"
                print(labels)
            
        with open(os.path.join(yolo_txt_dir, label_name), 'w') as f:
            f.write(labels)

for image in root.iter('image'): 루프 안에서 labels를 초기화하는 부분은 올바르게 구현. </br> 
for box in image.iter('box'): 루프 내부에서 발생하는 문제일 가능성. </br>
이 루프가 각 이미지에 대한 박스 정보만을 처리하고 있어야 하지만, 현재 결과를 보면 그렇지 않음.

### 코드 버전2 (에러 없음)

In [ ]:
if not os.path.exists(yolo_txt_dir):
    os.makedirs(yolo_txt_dir)

for file in os.listdir(xml_folder):
    # 경로에 있는 xml 파일을 읽어 파싱
    if file.endswith('.xml'):
        print(file)
        tree = ET.parse(os.path.join(xml_folder, file))
        # XML 문서의 구조는 요소들의 계층 구조로 표현되며, 이 중 최상위에 위치하는 요소를 루트라고 하는데, getroot() 메서드를 사용하여 해당 루트에 직접 접근
        root = tree.getroot()

       
        for image in root.iter('image'):  # tag가 이미지면, iter
            labels = ''  # 레이블 정보를 저장한 변수, 레이블 초기화
            # image 태그의 attrib에서 이름 추출
            image_name = image.attrib['name']
            label_name = image_name.replace('.jpg', '.txt')

            width = float(image.attrib['width'])  # image 태그의 attrib에서 가로
            height = float(image.attrib['height'])  # image 태그의 attrib에서 높이

            # for loop 반복문을 돌면서,
            for box in image.iter('box'):
                # 바운딩 박스 'box'태그에서 'label'의 값을 읽어오고, cls로 표현
                label = box.attrib['label']
                cls = get_class_index(label)

                xtl = float(box.attrib['xtl'])
                ytl = float(box.attrib['ytl'])
                xbr = float(box.attrib['xbr'])
                ybr = float(box.attrib['ybr'])

                x_center = (xtl + (xbr - xtl) / 2) / width
                y_center = (ytl + (ybr - ytl) / 2) / height
                box_w = (xbr - xtl) / width
                box_h = (ybr - ytl) / height

                labels += f"{cls} {x_center} {y_center} {box_w} {box_h}" + "\n"

            with open(os.path.join(yolo_txt_dir, label_name), 'w') as f:
                f.write(labels)

In [16]:
import os
import cv2 
import xml.etree.ElementTree as ET

def get_class_index(label):
    '''클래스 레이블을 클래스 인덱스로 매핑'''
    
    class_mapping = {'Hyoid_Bone': 0,
                     'Neck_Bone': 1, 'Food_Locate': 2, 'Food': 3}
    return class_mapping.get(label, 3)  # Default to 3 if label not found


def convert_to_YOLOformat(xml_folder, yolo_txt_dir):
    '''xml 파일에서 클래스별 바운딩박스 좌표를 YOLO 포맷으로 변환(레이블 데이터 가공)하는 함수'''

    if not os.path.exists(yolo_txt_dir):
        os.makedirs(yolo_txt_dir)

    for file in os.listdir(xml_folder):
        if file.endswith('.xml'):
            print(file)
            tree = ET.parse(os.path.join(xml_folder, file))
            root = tree.getroot()

            for image in root.iter('image'):
                labels = ''  # 레이블 초기화
                image_name = image.attrib['name']
                label_name = image_name.replace('.jpg', '.txt')

                width = float(image.attrib['width'])
                height = float(image.attrib['height'])

                for box in image.iter('box'):
                    label = box.attrib['label']
                    cls = get_class_index(label)

                    xtl = float(box.attrib['xtl'])
                    ytl = float(box.attrib['ytl'])
                    xbr = float(box.attrib['xbr'])
                    ybr = float(box.attrib['ybr'])

                    x_center = (xtl + (xbr - xtl) / 2) / width
                    y_center = (ytl + (ybr - ytl) / 2) / height
                    box_w = (xbr - xtl) / width
                    box_h = (ybr - ytl) / height

                    labels += f"{cls} {x_center} {y_center} {box_w} {box_h}\n"

                with open(os.path.join(yolo_txt_dir, label_name), 'w') as f:
                    f.write(labels)

### 코드 버전3 & XML Data Exploration

- https://wikidocs.net/127832
- https://medium.com/red-buffer/converting-a-custom-dataset-from-coco-format-to-yolo-format-6d98a4fd43fc 
- https://yolo-lolo.tistory.com/25 <br>
<br>
xml 을 딕셔너리로 변환 후 파싱

In [37]:
%pip install xmltodict

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\USER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [ ]:
import os
import xmltodict

def get_class_index(label):
    '''클래스 레이블을 클래스 인덱스로 매핑'''
    class_mapping = {'Hyoid_Bone': 0,
                     'Neck_Bone': 1, 'Food_Locate': 2, 'Food': 3}
    return class_mapping.get(label, 3)  # Default to 3 if label not found


xml_dir = './data/extract_img_test/extracted_img_231116_인덱싱에러수정(1)/25024694_현윤옥_(1)_YP_인덱싱수정'
output_folder = './data/yolo_label_test/labels_231127'


for filename in os.listdir(xml_dir):
    if filename.endswith(".xml"):
        with open(os.path.join(xml_dir, filename), "r") as f:
            data = xmltodict.parse(f.read())
            print(data['annotations'].keys())
            print(data['annotations']['image'])
            xml_img_data = data['annotations']['image']
            print(xml_img_data[0])
            print(xml_img_data[1])
            print("number of images", len(xml_img_data)) # 119개 이미지
            
            print("image name of id=0 is:", xml_img_data[0]['@name'])
            
            # 이미지 파일의 이름에서 확장자를 제거하여 txt 파일의 이름을 생성
            replace_extension = os.path.splitext(xml_img_data[0]['@name'])[0] + '.txt'
            print("replce extension:", replace_extension)
            file_path = os.path.join(output_folder, replace_extension)
            # output_folder가 없다면 만들기
            os.makedirs(output_folder, exist_ok=True)
        
            img_width = float(xml_img_data[0]['@width'])
            img_height = float(xml_img_data[0]['@height'])
            print("width of img id=0 is:", img_width)
            print("heigth of img id=0 is:",img_height)
            
            print("number of objects/bbox of img id=0 is:", len(xml_img_data[0]['box']))
            print(xml_img_data[0]['box'][0])
            
            
            with open(file_path, 'w') as file:
                for obj in xml_img_data[0]['box']:
                    class_name = obj['@label']
                    class_id = get_class_index(class_name)
                    print(class_name)
                    print(class_id)
                    
                    if class_id is not None:
                        xtl = float(obj['@xtl'])
                        ytl = float(obj['@ytl'])
                        xbr = float(obj['@xbr'])
                        ybr = float(obj['@ybr'])
                        
                        x_center = (xtl + (xbr - xtl) / 2) / img_width
                        y_center = (ytl + (ybr - ytl) / 2) / img_height
                        box_w = (xbr - xtl) / img_width
                        box_h = (ybr - ytl) / img_height
                        
                        file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}\n")
                    

In [ ]:
import os
import xmltodict

def get_class_index(label):
    '''클래스 레이블을 클래스 인덱스로 매핑'''
    class_mapping = {'Hyoid_Bone': 0,
                     'Neck_Bone': 1, 
                     'Food_Locate': 2, 
                     'Food': 3}
    return class_mapping.get(label, 3)  # Default to 3 if label not found


parent_dir = '/home/sh_rsrehab/AKAS_Test/data/extracted_img_231116_인덱싱에러수정(1)/'
output_folder = './data//yolo_label_test/labels_231204/'

for subdir in os.listdir(parent_dir):
    xml_directory = os.path.join(parent_dir, subdir)

    if os.path.isdir(xml_directory):

        for filename in os.listdir(xml_dir):
            if filename.endswith(".xml"):
                with open(os.path.join(xml_dir, filename), "r") as f:
                    data = xmltodict.parse(f.read())
                    xml_img_data = data['annotations']['image']

                    for img_data in xml_img_data: 
                        print("image name:", img_data['@name'])
                        # 이미지 파일의 이름에서 확장자를 제거하여 txt 파일의 이름을 생성
                        replace_extension = os.path.splitext(img_data['@name'])[0] + '.txt'
                        file_path = os.path.join(output_folder, replace_extension)
                        os.makedirs(output_folder, exist_ok=True)
                        
                        img_width = float(img_data['@width'])
                        img_height = float(img_data['@height'])

                        with open(file_path, 'w') as file:
                            for obj in img_data['box']:
                                class_name = obj['@label']
                                class_id = get_class_index(class_name)

                                if class_id is not None:
                                    xtl = float(obj['@xtl'])
                                    ytl = float(obj['@ytl'])
                                    xbr = float(obj['@xbr'])
                                    ybr = float(obj['@ybr'])

                                    x_center = (xtl + (xbr - xtl) / 2) / img_width
                                    y_center = (ytl + (ybr - ytl) / 2) / img_height
                                    box_w = (xbr - xtl) / img_width
                                    box_h = (ybr - ytl) / img_height

                                    file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}\n")